In [2]:
import datasets

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = datasets.load_dataset(
    'csv', 
    data_files='./data/ecg_unprocessed/30107.csv',
    features=datasets.Features({
        'timestamp': datasets.Value('string'),
        'signal': datasets.Value('float64'),
        'category': datasets.Value('string'),
    })
)['train']

In [4]:
import neurokit2 as nk
import pyhrv

In [5]:
sample = dataset[0:0+60 * 1000]

In [6]:
import scipy
from scipy import stats
import numpy as np

def TINN(x:np.array):
  """ Compute all the triangular interpolation to calculate the TINN scores. It also computes HRV index from an array x which contains 
      all the interbeats times for a given ECG signal.

      The axis is divided in 2 parts respectively on the right and left of the abscissa of the maximum value of the gaussian distribution
      The TINN score calculation is defined in the WESAD Dataset paper, to calculate it we needthe closest triangular interpolation 
      of the gaussian distribution of the interbeats times. The triangular interpolation is defined by 2 lines that meet at the maximum value
      of the gaussian distribution and cross the x-axis in N on the first half of the x-axis and M on the second half of the x-axis. 
      Thus inside ]N;M[ the interpolation function != 0
      Outside of ]N;M[ the interpolation function equals 0.
  """

  kernel = stats.gaussian_kde(x) #Create an approximated kernel for gaussian distribution from the x array (interbeats times)
  absi=np.linspace(np.min(x),np.max(x),len(x)) # Compute the x-axis of the interbeats distribution (from minimum interbeat time to maximum interbeat time)
  val=kernel.evaluate(absi) # Fit the gaussian distribution to the created x-axis
  ecart=absi[1]-absi[0] # Space between 2 values on the axis
  maxind=np.argmax(val) # Select the index for which the gaussian distribution (val array) is maximum 
  max_pos=absi[maxind]  # Interbeat time (abscissa) for which the gaussian distribution is maximum
  maxvalue=np.amax(val) # Max of the gaussian distribution
  N_abs=absi[0:maxind+1] # First half of the x-axis
  M_abs=absi[maxind:] # Second half of the x-axis
  HRVindex=len(x)/maxvalue
  err_N=[]
  err_M=[]

  for i in range(0,len(N_abs)-1):
    N=N_abs[i]
    slope=(maxvalue)/(max_pos-N)
    D=val[0:maxind+1]
    q=np.clip(slope*ecart*np.arange(-i,-i+maxind+1),0,None) #Triangular interpolation on the First half of the x-axis
    diff=D-q 
    err=np.multiply(diff,diff)
    err1=np.delete(err,-1)
    err2=np.delete(err, 0)
    errint=(err1+err2)/2
    errtot=np.linalg.norm(errint) # Error area between the triangular interpolation and the gaussian distribution on the first half of the x-axis
    err_N.append((errtot,N,N_abs,q))
  
  for i in range(1,len(M_abs)):
    M=M_abs[i]
    slope=(maxvalue)/(max_pos-M)
    D=val[maxind:]
    q=np.clip(slope*ecart*np.arange(-i,len(D)-i),0,None) #Triangular interpolation on the second half of the x-axis
    diff=D-q
    err=np.multiply(diff,diff)
    err1=np.delete(err,-1)
    err2=np.delete(err, 0)
    errint=(err1+err2)/2
    errtot=np.linalg.norm(errint) # Error area between the triangular interpolation and the gaussian distribution on the second half of the x-axis
    err_M.append((errtot,M,M_abs,q))

  return (err_N,err_M,absi,val,HRVindex)

def best_TINN(x:np.array):
  """Select the best N and M that give the best triangular interpolation function approximation of the gaussian distrbution and return
    N; M; the TINN score = M-N ; and the HRV index
  
  """
  err_N,err_M,_,_,HRVindex=TINN(x)
  N=np.argmin(np.array(err_N,dtype=object)[:,0])
  M=np.argmin(np.array(err_M,dtype=object)[:,0])
  absN=err_N[N][1]
  absM=err_M[M][1]
  return float(absN),float(absM),float(absM-absN),HRVindex

# _,_,T,HRVindex=best_TINN(hrv)
# T, HRVindex

In [7]:
peaks, _ = nk.ecg_peaks(sample['signal'], sampling_rate=1000)
peaks_indices = peaks[peaks['ECG_R_Peaks'] == 1].index

## HRV
hrv = np.array([(peaks_indices[i]-peaks_indices[i-1])/1000 for i in range(1,len(peaks_indices))])


In [9]:
TINN(hrv)

([(21.12408456637523,
   0.545,
   array([0.545     , 0.54876471, 0.55252941, 0.55629412, 0.56005882,
          0.56382353, 0.56758824, 0.57135294, 0.57511765, 0.57888235,
          0.58264706, 0.58641176, 0.59017647, 0.59394118, 0.59770588,
          0.60147059, 0.60523529, 0.609     , 0.61276471, 0.61652941,
          0.62029412, 0.62405882, 0.62782353, 0.63158824, 0.63535294,
          0.63911765, 0.64288235, 0.64664706, 0.65041176, 0.65417647,
          0.65794118, 0.66170588, 0.66547059, 0.66923529, 0.673     ,
          0.67676471, 0.68052941, 0.68429412, 0.68805882, 0.69182353,
          0.69558824, 0.69935294, 0.70311765, 0.70688235, 0.71064706]),
   array([0.        , 0.08527144, 0.17054289, 0.25581433, 0.34108578,
          0.42635722, 0.51162867, 0.59690011, 0.68217156, 0.767443  ,
          0.85271445, 0.93798589, 1.02325734, 1.10852878, 1.19380023,
          1.27907167, 1.36434312, 1.44961456, 1.53488601, 1.62015745,
          1.7054289 , 1.79070034, 1.87597179, 1.96124323

In [8]:
best_TINN(hrv)

(0.545, 0.865, 0.31999999999999995, 22.921453441914178)